In [ ]:
import math
import scipy
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import nltk
from nltk.corpus import stopwords

In [ ]:
# verified
df = pd.read_csv("VerifiedDataset.csv",on_bad_lines='skip')

# unverified
df2 = pd.read_csv("stackoverflow_full.csv",on_bad_lines='skip')

# jobs
linkedin_jobs = pd.read_csv("LinkedIn Jobs-Table 1.csv",on_bad_lines='skip')
geton_jobs = pd.read_csv("GetonBoard Jobs-Table 1.csv",on_bad_lines='skip')

In [ ]:
#===================Data Exploration and Cleaning====================

In [ ]:
print(df.columns)
print(df2.columns)

nan_count = df['HaveWorkedWith'].isna().sum()
print(f'nan count for verified data {nan_count}')

nan_count2 = df2["HaveWorkedWith"].isna().sum()
print(f'nan count for unverified data {nan_count2}')

Index(['Unnamed: 0', 'Age', 'Accessibility', 'EdLevel', 'Employment', 'Gender',
       'MentalHealth', 'MainBranch', 'YearsCode', 'YearsCodePro', 'Country',
       'PreviousSalary', 'HaveWorkedWith', 'ComputerSkills', 'Employed',
       'Unnamed: 15'],
      dtype='object')
Index(['Unnamed: 0', 'Age', 'Accessibility', 'EdLevel', 'Employment', 'Gender',
       'MentalHealth', 'MainBranch', 'YearsCode', 'YearsCodePro', 'Country',
       'PreviousSalary', 'HaveWorkedWith', 'ComputerSkills', 'Employed'],
      dtype='object')
nan count for verified data 63
nan count for unverified data 63


In [ ]:
df = df.dropna(subset=['HaveWorkedWith'])
df2 = df.dropna(subset=['HaveWorkedWith'])
nan_count = df["HaveWorkedWith"].isna().sum()
nan_count2 = df2["HaveWorkedWith"].isna().sum()

print(f'nan count for verified data {nan_count}')
print(f'nan count for unverified data {nan_count2}')

nan count for verified data 0
nan count for unverified data 0


In [ ]:
print(linkedin_jobs.columns)
print(geton_jobs.columns)

Index(['company_name', 'title', 'description', 'formatted_work_type',
       'job_posting_url', 'application_url', 'formatted_experience_level',
       'skills_desc'],
      dtype='object')
Index(['Source', 'Title', 'Company', 'Description', 'Link'], dtype='object')


In [ ]:
# Parsing!!!

In [ ]:
#  Dataset before
linkedin_jobs.head(2)

,company_name,title,description,formatted_work_type,job_posting_url,application_url,formatted_experience_level,skills_desc
0,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,Full-time,https://www.linkedin.com/jobs/view/921716/?trk...,NaN,NaN,Requirements: \n\nWe are seeking a College or ...
1,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",Full-time,https://www.linkedin.com/jobs/view/1829192/?tr...,NaN,NaN,NaN


In [ ]:
# Dataset before
geton_jobs.head(2)

,Source,Title,Company,Description,Link
0,Get on Board,Cloud Engineer,CMPC,MISIÓN\nLa infraestructura Cloud para CMPC cor...,https://www.getonbrd.com/jobs/sysadmin-devops-...
1,Get on Board,Desarrollador Cloud,Dirección ChileCompra,Somos la institución encargada de administrar ...,https://www.getonbrd.com/jobs/sysadmin-devops-...


In [ ]:
# stopwords english version
nltk.download('stopwords')
stop_words_en = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# parsing for mostly english text
def parse_text_linkedin(text):
  text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text) # any joint words
  text = text.lower() # lowercase
  text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # non-ASCII characters
  text = re.sub(r'\d+', '', text)  # rid of all #
  #text = re.findall(r'\b[a-zA-Z0-9\'-]+\b', text)
  words = re.findall(r'\b[a-zA-Z\'-]+\b', text)  #  keep alphabetic characters and hyphenated words
  words = [word for word in words if word not in stop_words_en] # remove stopwords english ver
  text = ' '.join(words)
  text = re.sub(r'\s+', ' ', text).strip()
  return text

# example to see if it works on english text
text_main = "€™s orders. â€¢ 34 Read physiciansâ€™ orders, measure arterial blood gases, and review patient information to assess patientâ€™s condition. â€¢ Explain treatment procedures to patients to gain cooperation and allay fears. â€¢ Monitor patientâ€™s physiological responses to therapy such as vital signs, arterial blood gases and blood chemistry changes and consult with physician if adverse reactions occur. â€¢ Administer therapeutic gases including nitrogen, nitric oxide, heliox, etc. â€¢ Enforce safety rules and ensure careful adherence to physiciansâ€™ orders. â€¢ Maintain charts that contain patient pertinent identification and therapy information. â€¢ Inspect, clean, test, and maintain respiratory therapy equipment to ensure equipment is functioning safely and efficiently and notify manager/supervisor when repairs are necessary. â€¢"

parse_text_linkedin(text_main)

'orders read physicians orders measure arterial blood gases review patient information assess patient condition explain treatment procedures patients gain cooperation allay fears monitor patient physiological responses therapy vital signs arterial blood gases blood chemistry changes consult physician adverse reactions occur administer therapeutic gases including nitrogen nitric oxide heliox etc enforce safety rules ensure careful adherence physicians orders maintain charts contain patient pertinent identification therapy information inspect clean test maintain respiratory therapy equipment ensure equipment functioning safely efficiently notify manager supervisor repairs necessary'

In [ ]:
# applying the parse_text function to the description column LINKEDIN DATASET
linkedin_jobs['description'] = linkedin_jobs['description'].apply(lambda x: parse_text_linkedin(str(x)) if x is not None else "")

# show the updated dataframe
print(linkedin_jobs[['company_name','description']])

                 company_name  \
0       Corcoran Sawyer Smith   
1                         NaN   
2      The National Exemplar    
3      Abrams Fensterman, LLP   
4                         NaN   
..                        ...   
704       Stiles CFO Services   
705       Kaizen Technologies   
706  Positive Masculinity Now   
707              Diverse Lynx   
708   Harmony Healthcare Inc.   

                                           description  
0    job description leading real estate firm new j...  
1    aspen therapy wellness committed serving clien...  
2    national exemplar accepting applications assis...  
3    senior associate attorney elder law trusts est...  
4    looking hvac service tech experience commerica...  
..                                                 ...  
704  company description fractional cfo offer full ...  
705  help desk support illinois street suite golden...  
706  public relations specialist wanted positive ma...  
707  job title architect location

In [ ]:
# stopwords spanish version
nltk.download('stopwords')
stop_words_spa = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# # spanish data set parse
# def parse_text_getonjobs(text):
#   text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text) # any joint words
#   text = text.lower() # lowercase
#   text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # non-ASCII characters
#   text = re.sub(r'\d+', '', text)  # rid of all numbers
#   words = re.findall(r'\b[a-zA-Z0-9\'-]+\b', text)
#   #words = re.findall(r'\b[a-zA-Z\'-]+\b', text)  #  keep alphabetic characters and hyphenated words
#   words = [word for word in words if word not in stop_words_spa] # remove stopwords spanish ver
#   text = ' '.join(words)
#   text = re.sub(r'\s+', ' ', text).strip()
#   return text

# # example to see if it works
# text_main_s = "â€¢ to and 67 allay fears. a to gases and MISIÓN la infraestructura Cloud para CMPC corresponde el stack tecnológico diferenciador que permitirá a la compañía alcanzar las metas de Innovación de acuerdo a la estrategia que acordó el directorio. Por lo cual, el Cloud Engineer deberá administrar, gestionar y controlar los contratos de administración Cloud directos e indirectos que CMPC posea con proveedores, velando por el adecuado cumplimiento de SLA´s y asegurando que el servicio entregado sea de excelencia y calidad; Así como administrar los diferentes Cloud provider que posea CMPC, eones, yaicios gestionados con terceros"

# parse_text_getonjobs(text_main_s)




# FIXED VER
# spanish data set parse
def parse_text_getonjobs(text):
  text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text) # any joint words
  text = text.lower() # lowercase
  text = re.sub(r'\d+', '', text)  # rid of all numbers

  words = re.findall(r'\b[a-záéíóúñ0-9\'-]+\b', text)  # kleep hyphenated words
  words = [word for word in words if word not in stop_words_spa] # remove spanish stop words
  text = ' '.join(words)
  text = re.sub(r'\s+', ' ', text).strip()
  return text

# example to see if it works
text_main_s = "â€¢  to and allay fears. a to od gases and MISIÓN la infraestructura Cloud para CMPC corresponde el stack tecnológico diferenciador que permitirá a la compañía alcanzar las metas de Innovación de acuerdo a la estrategia que acordó el directorio. Por lo cual, el Cloud Engineer deberá administrar, gestionar y controlar los contratos de administración Cloud directos e indirectos que CMPC posea con proveedores, velando por el adecuado cumplimiento de SLA´s y asegurando que el servicio entregado sea de excelencia y calidad; Así como administrar los diferentes Cloud provider que posea CMPC, eones, yaicios gestionados con terceros"

parse_text_getonjobs(text_main_s)

'to and allay fears to od gases and misión infraestructura cloud cmpc corresponde stack tecnológico diferenciador permitirá compañía alcanzar metas innovación acuerdo estrategia acordó directorio cloud engineer deberá administrar gestionar controlar contratos administración cloud directos indirectos cmpc posea proveedores velando adecuado cumplimiento sla s asegurando servicio entregado excelencia calidad así administrar diferentes cloud provider posea cmpc eones yaicios gestionados terceros'

In [ ]:
# applying the parse_text function to the description column GETON_JOBS DATASET
geton_jobs['Description'] = geton_jobs['Description'].apply(lambda x: parse_text_getonjobs(str(x)) if x is not None else "")

# show the updated dataframe
print(geton_jobs[['Company','Description']])

                  Company                                        Description
0                    CMPC  misión infraestructura cloud cmpc corresponde ...
1   Dirección ChileCompra  institución encargada administrar plataforma t...
2       Sophos Solutions   acerca empleo sophos gft compañía global soluc...
3       Sophos Solutions   compañía global soluciones tecnológicas operac...
4       Digitalproserver   digitalproserver desarrollamos soluciones espe...
..                    ...                                                ...
95                   TCIT  misiones tcit descubrir potenciar mejores tale...
96                   TCIT  misiones tcit descubrir potenciar mejores tale...
97                   TCIT  misiones tcit descubrir potenciar mejores tale...
98                   TCIT  misiones tcit descubrir potenciar mejores tale...
99                2BRAINS  brains empresa dedicada construir desarrollar ...

[100 rows x 2 columns]


In [ ]:
# after linkedin_jobs
linkedin_jobs.head(2)

,company_name,title,description,formatted_work_type,job_posting_url,application_url,formatted_experience_level,skills_desc
0,Corcoran Sawyer Smith,Marketing Coordinator,job description leading real estate firm new j...,Full-time,https://www.linkedin.com/jobs/view/921716/?trk...,NaN,NaN,Requirements: \n\nWe are seeking a College or ...
1,NaN,Mental Health Therapist/Counselor,aspen therapy wellness committed serving clien...,Full-time,https://www.linkedin.com/jobs/view/1829192/?tr...,NaN,NaN,NaN


In [ ]:
# after geton_jobs
geton_jobs.head(2)

,Source,Title,Company,Description,Link
0,Get on Board,Cloud Engineer,CMPC,misión infraestructura cloud cmpc corresponde ...,https://www.getonbrd.com/jobs/sysadmin-devops-...
1,Get on Board,Desarrollador Cloud,Dirección ChileCompra,institución encargada administrar plataforma t...,https://www.getonbrd.com/jobs/sysadmin-devops-...


In [ ]:
#===================TF-IDF Embedding=====================

In [ ]:
#TFIDF Encoding on verified dataset
vectorizer = TfidfVectorizer()

all_text = pd.concat([df["HaveWorkedWith"],df2["HaveWorkedWith"], linkedin_jobs["description"], geton_jobs["Description"]]).dropna()
vectorizer.fit(all_text)

# Transform each dataset separately using the same vectorizer
verified_tfidf = vectorizer.transform(df["HaveWorkedWith"])
unverified_tfidf = vectorizer.transform(df2["HaveWorkedWith"])

linkedin_jobs_tfidf = vectorizer.transform(linkedin_jobs["description"])
geton_jobs_tfidf = vectorizer.transform(geton_jobs["Description"])

In [ ]:
#TFIDF Encoding on Jobs Datasets

# Transform each dataset separately using the same vectorizer
verified_tfidf = vectorizer.transform(df["HaveWorkedWith"])
unverified_tfidf = vectorizer.transform(df2["HaveWorkedWith"])
linkedin_jobs_tfidf = vectorizer.transform(linkedin_jobs["description"])
geton_jobs_tfidf = vectorizer.transform(geton_jobs["Description"])

In [ ]:
from os import getresuid
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#NEED TO RESHAPE SINCE DATASETS ARE NOT COMPATIBLE SHAPES

#Compute cosine similarity between candidates and LinkedIn job descriptions
sim_linkedin_ver = cosine_similarity(verified_tfidf, linkedin_jobs_tfidf)
print(sim_linkedin_ver)
sim_linkedin_unver = cosine_similarity(unverified_tfidf, linkedin_jobs_tfidf)
print(sim_linkedin_unver)

[[0.00278625 0.         0.         ... 0.00531421 0.00869616 0.0019325 ]
 [0.00265308 0.         0.         ... 0.0024371  0.01799048 0.0013065 ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.00530813 0.         0.         ... 0.00530487 0.         0.        ]
 [0.00585403 0.         0.         ... 0.00323319 0.00637361 0.00063496]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[[0.00278625 0.         0.         ... 0.00531421 0.00869616 0.0019325 ]
 [0.00265308 0.         0.         ... 0.0024371  0.01799048 0.0013065 ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.00530813 0.         0.         ... 0.00530487 0.         0.        ]
 [0.00585403 0.         0.         ... 0.00323319 0.00637361 0.00063496]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
# Compute cosine similarity between candidates and GetOn job descriptions
sim_geton_ver = cosine_similarity(verified_tfidf, geton_jobs_tfidf)
print(sim_geton_ver)
sim_geton_unver = cosine_similarity(unverified_tfidf, geton_jobs_tfidf)
print(sim_geton_unver)

[[0.02670853 0.00353162 0.00201202 ... 0.00541829 0.00642327 0.0096993 ]
 [0.00706578 0.         0.00660594 ... 0.00264847 0.00659127 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.08288721 0.01096002 0.00156028 ... 0.00987041 0.00790861 0.02028035]
 [0.03608234 0.00423278 0.00293871 ... 0.00995806 0.00547765 0.01035995]
 [0.01178215 0.         0.00220457 ... 0.00254921 0.         0.00568841]]
[[0.02670853 0.00353162 0.00201202 ... 0.00541829 0.00642327 0.0096993 ]
 [0.00706578 0.         0.00660594 ... 0.00264847 0.00659127 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.08288721 0.01096002 0.00156028 ... 0.00987041 0.00790861 0.02028035]
 [0.03608234 0.00423278 0.00293871 ... 0.00995806 0.00547765 0.01035995]
 [0.01178215 0.         0.00220457 ... 0.00254921 0.         0.00568841]]


In [ ]:
# For each candidate, find the best match in LinkedIn
bst_mtch_linkedin_ver = sim_linkedin_ver.argmax(axis=1)
print(bst_mtch_linkedin_ver)
bst_mtch_linkedin_unver = sim_linkedin_unver.argmax(axis=1)
print(bst_mtch_linkedin_unver)

[163 562 547 ... 663 602 163]
[163 562 547 ... 663 602 163]


In [ ]:
# For each candidate, find the best match in GetOn
bst_mtch_geton_ver = sim_geton_ver.argmax(axis=1)
print(bst_mtch_geton_ver)
bst_mtch_geton_unver = sim_geton_unver.argmax(axis=1)
print(bst_mtch_geton_unver)

[ 0 93 93 ...  0  5 80]
[ 0 93 93 ...  0  5 80]


In [ ]:
#===================Results=====================

In [ ]:
#===================LINKEDIN DATASET=======================

In [ ]:
import numpy as np

most_similar_li = np.argmax(sim_linkedin_ver, axis=1)
print("Most similar LinkedIn job for each verified candidate:")
print(most_similar_li)

highest_similarity_li = np.max(sim_linkedin_ver, axis=1)
print("Highest similarity values:")
print(highest_similarity_li)

Most similar LinkedIn job for each verified candidate:
[163 562 547 ... 663 602 163]
Highest similarity values:
[0.04812734 0.06531957 0.01324304 ... 0.04679135 0.03280919 0.03796596]


In [ ]:
print(most_similar_li)
linkedin_jobs_og = pd.read_csv("LinkedIn Jobs-Table 1.csv",on_bad_lines='skip')

most_similar_li_job_titles = df.iloc[most_similar_li]['HaveWorkedWith'].values
print(most_similar_li_job_titles)

[163 562 547 ... 663 602 163]
['Assembly;C;Go;HTML/CSS;Java;JavaScript;Node.js;SQL;Docker;Git;Yarn;Angular.js;Express;jQuery;Spring;Vue.js;DigitalOcean;Heroku;Elasticsearch;MariaDB;MongoDB;MySQL;Oracle'
 'C;HTML/CSS;JavaScript;Python;SQL;TypeScript;Docker;npm;Yarn;Express;Next.js;Node.js;React.js;MongoDB;MySQL;PostgreSQL;Redis'
 'Bash/Shell;C;C++;Java;Julia;Python;Docker;Git;Django;Flask;AWS;Microsoft Azure;PostgreSQL;SQLite'
 ...
 'HTML/CSS;Java;JavaScript;Kotlin;Node.js;SQL;TypeScript;Docker;Git;Kubernetes;Angular;Express;Spring;Firebase;MongoDB;SQLite'
 'Bash/Shell;C#;JavaScript;Node.js;PowerShell;SQL;TypeScript;Docker;Git;Terraform;ASP.NET;ASP.NET Core ;React.js;AWS;Microsoft SQL Server;MySQL'
 'Assembly;C;Go;HTML/CSS;Java;JavaScript;Node.js;SQL;Docker;Git;Yarn;Angular.js;Express;jQuery;Spring;Vue.js;DigitalOcean;Heroku;Elasticsearch;MariaDB;MongoDB;MySQL;Oracle']


In [ ]:
# [Verified] Display the best job for each candidate along with the similarity score

best_matches_li = np.argmax(sim_linkedin_ver, axis=1)
print(best_matches_li)

for i, match in enumerate(best_matches_li):

  if sim_linkedin_ver[i][match] > .15:
    print(f"{i}, {match}")

    print(f"Candidate {i} - {linkedin_jobs.loc[match, 'title']} with a similarity score of {sim_linkedin_ver[i][match]:.2f}")



[163 562 547 ... 663 602 163]
2185, 602
Candidate 2185 - AWS Cloud Architect (W2 Only) with a similarity score of 0.17
2413, 602
Candidate 2413 - AWS Cloud Architect (W2 Only) with a similarity score of 0.19
2488, 602
Candidate 2488 - AWS Cloud Architect (W2 Only) with a similarity score of 0.17
2500, 602
Candidate 2500 - AWS Cloud Architect (W2 Only) with a similarity score of 0.17
2648, 602
Candidate 2648 - AWS Cloud Architect (W2 Only) with a similarity score of 0.19
2739, 163
Candidate 2739 - Java architect / Lead Java developer with a similarity score of 0.16
4294, 163
Candidate 4294 - Java architect / Lead Java developer with a similarity score of 0.16
4889, 163
Candidate 4889 - Java architect / Lead Java developer with a similarity score of 0.16
5863, 602
Candidate 5863 - AWS Cloud Architect (W2 Only) with a similarity score of 0.17
5877, 602
Candidate 5877 - AWS Cloud Architect (W2 Only) with a similarity score of 0.16
6061, 602
Candidate 6061 - AWS Cloud Architect (W2 Only) wi

In [ ]:
# [Unverified] Display the best job for each candidate along with the similarity score

best_matches_li_unver = np.argmax(sim_linkedin_unver, axis=1)
print(best_matches_li_unver)

for i, match in enumerate(best_matches_li_unver):

  if sim_linkedin_unver[i][match] > .10:
    print(f"{i}, {match}")

    print(f"Candidate {i} - {linkedin_jobs.loc[match, 'title']} with a similarity score of {swim_linkedin_unver[i][match]:.2f}")



[163 562 547 ... 663 602 163]
12, 602
Candidate 12 - AWS Cloud Architect (W2 Only) with a similarity score of 0.10
49, 398
Candidate 49 - Search Engine Optimization Specialist with a similarity score of 0.13
154, 602
Candidate 154 - AWS Cloud Architect (W2 Only) with a similarity score of 0.11
192, 602
Candidate 192 - AWS Cloud Architect (W2 Only) with a similarity score of 0.11
226, 602
Candidate 226 - AWS Cloud Architect (W2 Only) with a similarity score of 0.13
227, 602
Candidate 227 - AWS Cloud Architect (W2 Only) with a similarity score of 0.14
314, 398
Candidate 314 - Search Engine Optimization Specialist with a similarity score of 0.10
390, 163
Candidate 390 - Java architect / Lead Java developer with a similarity score of 0.13
391, 398
Candidate 391 - Search Engine Optimization Specialist with a similarity score of 0.12
486, 602
Candidate 486 - AWS Cloud Architect (W2 Only) with a similarity score of 0.11
521, 602
Candidate 521 - AWS Cloud Architect (W2 Only) with a similarity 

In [ ]:
#checking if matches are the same for both unver and ver
differences_li = best_matches_li - best_matches_li_unver
differences_geton = [x for x in differences_li if x != 0]
print(differences_geton)

[]


In [ ]:
#comparing candidate skills and job title match
for i, match in enumerate(best_matches_li):
  if sim_linkedin_ver[i][match] > .10:
    print(f"Candidate Skills:\t {df.loc[i, 'HaveWorkedWith']}")
    print(f"Job: \t\t\t {linkedin_jobs_og.loc[match, 'title']}")


Candidate Skills:	 Bash/Shell;C#;Crystal;HTML/CSS;JavaScript;Node.js;PHP;PowerShell;Python;SQL;VBA;Docker;Git;Terraform;Unity 3D;ASP.NET;ASP.NET Core ;Flask;jQuery;Laravel;React.js;AWS;Google Cloud Platform;Heroku;IBM Cloud or Watson;Microsoft Azure;Oracle Cloud Infrastructure;Cassandra;Firebase;MariaDB;Microsoft SQL Server;MongoDB;MySQL
Job: 			 AWS Cloud Architect (W2 Only)
Candidate Skills:	 C++;HTML/CSS;JavaScript;SQL;Unreal Engine;React.js;PostgreSQL;Redis
Job: 			 Search Engine Optimization Specialist
Candidate Skills:	 Assembly;Bash/Shell;C++;HTML/CSS;JavaScript;Matlab;Node.js;PowerShell;Python;SQL;TypeScript;Docker;Git;Angular;Angular.js;Express;React.js;Google Cloud Platform;Heroku;Oracle Cloud Infrastructure;Firebase;MySQL;PostgreSQL
Job: 			 AWS Cloud Architect (W2 Only)
Candidate Skills:	 C;C++;HTML/CSS;Java;JavaScript;Node.js;Python;R;SQL;TypeScript;Docker;Git;Angular;Angular.js;Django;Express;Flask;Oracle Cloud Infrastructure;Firebase;MongoDB
Job: 			 AWS Cloud Architect 

In [ ]:
#===================GETON DATASET=======================

In [ ]:
most_similar_geton = np.argmax(sim_geton_ver, axis=1)
print("Most similar Geton job for each verified candidate:")
print(most_similar_geton)

highest_similarity_geton = np.max(sim_geton_ver, axis=1)
print("Highest similarity values:")
print(highest_similarity_geton)

Most similar Geton job for each verified candidate:
[ 0 93 93 ...  0  5 80]
Highest similarity values:
[0.02670853 0.03105247 0.00949962 ... 0.08288721 0.04599713 0.02707361]


In [ ]:
print(most_similar_geton)
geton_jobs_og = pd.read_csv("GetonBoard Jobs-Table 1.csv", on_bad_lines='skip')

most_similar_geton_job_titles = df.iloc[most_similar_geton]['HaveWorkedWith'].values
print(most_similar_geton_job_titles)

[ 0 93 93 ...  0  5 80]
['Assembly;Bash/Shell;C;C#;C++;Dart;Elixir;Go;Haskell;HTML/CSS;Java;JavaScript;Node.js;PHP;PowerShell;Python;Rust;SQL;TypeScript;Deno;Docker;Git;Yarn;Django;Express;FastAPI;Flask;jQuery;React.js;Spring;Svelte;Vue.js;Google Cloud Platform;Heroku;MariaDB;Microsoft SQL Server;MongoDB;MySQL;PostgreSQL;SQLite'
 'C++;Java;PHP;Unity 3D;Google Cloud Platform;Heroku;Firebase;MongoDB'
 'C++;Java;PHP;Unity 3D;Google Cloud Platform;Heroku;Firebase;MongoDB' ...
 'Assembly;Bash/Shell;C;C#;C++;Dart;Elixir;Go;Haskell;HTML/CSS;Java;JavaScript;Node.js;PHP;PowerShell;Python;Rust;SQL;TypeScript;Deno;Docker;Git;Yarn;Django;Express;FastAPI;Flask;jQuery;React.js;Spring;Svelte;Vue.js;Google Cloud Platform;Heroku;MariaDB;Microsoft SQL Server;MongoDB;MySQL;PostgreSQL;SQLite'
 'Assembly;C;C++;Delphi;Go;Haskell;HTML/CSS;Java;JavaScript;Matlab;Node.js;PHP;Python;R;SQL;TypeScript;Ansible;Docker;Git;Kubernetes;Unity 3D;Yarn;Angular;Angular.js;Django;Flask;React.js;Vue.js;DigitalOcean;Google C

In [ ]:
# [Verified] Display the best job for each candidate along with the similarity score

best_matches_geton = np.argmax(sim_geton_ver, axis=1)
print(geton_jobs.shape)


for i, match in enumerate(best_matches_geton):
  if sim_geton_ver[i][match] > .10:
    print(f"Candidate {i} - {geton_jobs.loc[match, 'Title']} with a similarity score of {sim_geton_ver[i][match]:.2f}")


(100, 5)
Candidate 2371 - Cloud Engineer with a similarity score of 0.12
Candidate 2500 - Cloud Engineer with a similarity score of 0.11
Candidate 2560 - Cloud Engineer with a similarity score of 0.11
Candidate 2600 - Cloud Engineer with a similarity score of 0.12
Candidate 2633 - Cloud Engineer with a similarity score of 0.10
Candidate 2690 - Cloud Engineer with a similarity score of 0.11
Candidate 2691 - Cloud Engineer with a similarity score of 0.11
Candidate 2726 - Cloud Engineer with a similarity score of 0.15
Candidate 3448 - Cloud Engineer with a similarity score of 0.10
Candidate 5004 - Cloud Engineer with a similarity score of 0.10
Candidate 5189 - Cloud Engineer with a similarity score of 0.12
Candidate 5205 - Cloud Engineer with a similarity score of 0.11
Candidate 5430 - Cloud Engineer with a similarity score of 0.11
Candidate 5456 - Cloud Engineer with a similarity score of 0.11
Candidate 5621 - Cloud Engineer with a similarity score of 0.11
Candidate 5942 - Cloud Engineer

In [ ]:
# [Unverified] Display the best job for each candidate along with the similarity score

best_matches_geton_unver = np.argmax(sim_geton_unver, axis=1)
print(geton_jobs.shape)

for i, match in enumerate(best_matches_geton_unver):
  if sim_geton_unver[i][match] > .10:
    print(f"Candidate {i} - {geton_jobs.loc[match, 'Title']} with a similarity score of {sim_geton_unver[i][match]:.2f}")


(100, 5)
Candidate 2371 - Cloud Engineer with a similarity score of 0.12
Candidate 2500 - Cloud Engineer with a similarity score of 0.11
Candidate 2560 - Cloud Engineer with a similarity score of 0.11
Candidate 2600 - Cloud Engineer with a similarity score of 0.12
Candidate 2633 - Cloud Engineer with a similarity score of 0.10
Candidate 2690 - Cloud Engineer with a similarity score of 0.11
Candidate 2691 - Cloud Engineer with a similarity score of 0.11
Candidate 2726 - Cloud Engineer with a similarity score of 0.15
Candidate 3448 - Cloud Engineer with a similarity score of 0.10
Candidate 5004 - Cloud Engineer with a similarity score of 0.10
Candidate 5189 - Cloud Engineer with a similarity score of 0.12
Candidate 5205 - Cloud Engineer with a similarity score of 0.11
Candidate 5430 - Cloud Engineer with a similarity score of 0.11
Candidate 5456 - Cloud Engineer with a similarity score of 0.11
Candidate 5621 - Cloud Engineer with a similarity score of 0.11
Candidate 5942 - Cloud Engineer

In [ ]:
#checking if matches are the same for both unver and ver
differences_geton = best_matches_geton - best_matches_geton_unver
differences_geton = [x for x in differences_geton if x != 0]
print(differences_geton)

[]


In [ ]:
# comparing candidate skills and job title match
for i, match in enumerate(best_matches_geton):
  if sim_geton_ver[i][match] > .12:
    print(f"Candidate Skills:\t {df.loc[i, 'HaveWorkedWith']}")
    print(f"Job: \t\t\t {geton_jobs.loc[match, 'Title']}")

Candidate Skills:	 Java;Docker;Firebase;Google Cloud;Cloud Firestore;MongoDB
Job: 			 Cloud Engineer
Candidate Skills:	 C++;Python;Docker;Google Cloud
Job: 			 Cloud Engineer
Candidate Skills:	 R;Google Cloud
Job: 			 Cloud Engineer
Candidate Skills:	 Assembly;C;C#;C++;HTML/CSS;JavaScript;TypeScript;Docker;npm;Express;Node.js;React.js
Job: 			 Cloud Engineer
Candidate Skills:	 HTML/CSS;JavaScript;TypeScript;Git
Job: 			 Cloud Engineer
Candidate Skills:	 Dart;Go;HTML/CSS;JavaScript;PHP;TypeScript;npm;Yarn;Angular;Deno;Express;Node.js;React.js;Svelte;AWS;Firebase;Heroku;Cloud Firestore;MongoDB;MySQL;Firebase Realtime Database;Redis;SQLite
Job: 			 Cloud Engineer
Candidate Skills:	 Python;SQL;Firebase;Google Cloud;Microsoft Azure;MongoDB;MySQL
Job: 			 Cloud Engineer
Candidate Skills:	 C#;PowerShell;Microsoft SQL Server
Job: 			 Cloud Engineer
Candidate Skills:	 Java;Python;Git;Angular;AWS;PostgreSQL
Job: 			 Cloud Engineer
Candidate Skills:	 Java;SQL;Microsoft Azure;Microsoft SQL Server


In [ ]:
!python --version

Python 3.10.12
